In [1]:
#import lib
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import time
import json

In [2]:
#to append url with specific years
def append_url(url,tahun):
    url = url + tahun
    return url

In [3]:
#to get content of web
def get_web_content(url):
    headers = {'user-agent' : 'Mozilla/5.0 (X11; Linux x86_64); Vivianni/19vivianni@gmail.com'}
    r = requests.get(url, headers=headers) 
    while r.status_code != 200:
        r = requests.get(url, headers=headers) 
    soup = bs(r.content, 'html5lib') 
    table = soup.find_all('table')
    table = table[0]
    return table

In [4]:
#to get header of the web content
def get_header(table):
    data_row=['Tahun']
    for header in table.find_all('th'):
        data_row.append(header.get_text())
    return data_row

In [5]:
#to get table data from web content
def get_data(table,data,tahun):
    for row in table.find_all('tr'):
        data_row=[tahun]
        for cell in row.find_all('td'):
            data_row.append(cell.get_text())
        if (len(data_row)==1): #pass header
            pass
        else :
            data.append(data_row)
    return data

In [6]:
def content(url):
    data = []
    for i in range (2010,2019):
        URL = append_url(url, str(i))
        table = get_web_content(URL)
        data = get_data(table,data,str(i))
        time.sleep(1)
    
    #get the header
    header = get_header(table)
    
    #convert to dict
    final = []
    for i in range (0,len(data)):
        final.append(dict([(header[j],data[i][j]) for j in range (0,len(data[0]))]))
    return final

In [7]:
#main
URL_expor = "https://www.bps.go.id/tabel/tblbln-frameee.php?sumber1=1&thn1="
URL_impor = "https://www.bps.go.id/tabel/tblbln-frameee.php?sumber1=2&thn1="
expor = content(URL_expor)
impor = content(URL_impor)

In [8]:
# remove total from table
for data in expor:
    if (data['Bulan/Month']=='T  O  T  A  L'):
        expor.remove(data)
for data in impor:
    if (data['Bulan/Month']=='T  O  T  A  L'):
        impor.remove(data)

In [9]:
#to save data in json format in data directory
with open('../data/export.json', 'w') as json_file:  
    json.dump(expor, json_file)

with open('../data/import.json', 'w') as json_file:  
    json.dump(impor, json_file)